In [1]:
import pandas as pd
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl

In [2]:
os.chdir('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Repositório Local\\\\MUNIC')


In [31]:
df = pd.read_excel('Base_MUNIC_2019.xlsx', sheet_name='Governança')
df

,CodMun,REGIAO,COD UF,UF,NOME MUNIC,CLASSE POP,MGOV01,MGOV011A,MGOV011B,MGOV0121,...,MGOV303,MGOV31,MGOV3111,MGOV3112,MGOV3113,MGOV3114,MGOV3115,MGOV3116,MGOV3117,MGOV3118
0,1100015,1 - Norte,11,Rondônia,Alta Floresta D'Oeste,4 - 20001 até 50000,Sim,9639,2017,Sim,...,-,-,-,-,-,-,-,-,-,-
1,1100023,1 - Norte,11,Rondônia,Ariquemes,6 - 100001 até 500000,Sim,13460,2017,Não,...,-,-,-,-,-,-,-,-,-,-
2,1100031,1 - Norte,11,Rondônia,Cabixi,2 - 5001 até 10000,Sim,148,2016,Não,...,-,-,-,-,-,-,-,-,-,-
3,1100049,1 - Norte,11,Rondônia,Cacoal,5 - 50001 até 100000,Não,-,-,-,...,-,-,-,-,-,-,-,-,-,-
4,1100056,1 - Norte,11,Rondônia,Cerejeiras,3 - 10001 até 20000,Sim,7721,2012,Sim,...,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,5222005,5 - Centro-Oeste,52,Goiás,Vianópolis,3 - 10001 até 20000,Não,-,-,-,...,-,-,-,-,-,-,-,-,-,-
5566,5222054,5 - Centro-Oeste,52,Goiás,Vicentinópolis,2 - 5001 até 10000,Não,-,-,-,...,-,-,-,-,-,-,-,-,-,-
5567,5222203,5 - Centro-Oeste,52,Goiás,Vila Boa,2 - 5001 até 10000,Não,-,-,-,...,-,-,-,-,-,-,-,-,-,NaN
5568,5222302,5 - Centro-Oeste,52,Goiás,Vila Propício,2 - 5001 até 10000,Não,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [32]:
df = df[['CodMun','NOME MUNIC','COD UF', 'MGOV01']]
df

,CodMun,NOME MUNIC,COD UF,MGOV01
0,1100015,Alta Floresta D'Oeste,11,Sim
1,1100023,Ariquemes,11,Sim
2,1100031,Cabixi,11,Sim
3,1100049,Cacoal,11,Não
4,1100056,Cerejeiras,11,Sim
...,...,...,...,...
5565,5222005,Vianópolis,52,Não
5566,5222054,Vicentinópolis,52,Não
5567,5222203,Vila Boa,52,Não
5568,5222302,Vila Propício,52,Não


In [34]:
df.columns

Index(['CodMun', 'NOME MUNIC', 'COD UF', 'MGOV01'], dtype='object')

In [35]:
cod_uf = pd.read_csv('MUNIC_educacao_2021.csv', sep=';')[['UF','Cod UF']]


In [36]:
x= cod_uf.pivot_table(columns=('UF','Cod UF'), aggfunc='size')


In [37]:
cod_uf = pd.DataFrame(x).reset_index()[['UF','Cod UF']]

In [40]:
df = df.merge(cod_uf, right_on='Cod UF',left_on='COD UF')
df

,CodMun,NOME MUNIC,COD UF,MGOV01,UF,Cod UF
0,1100015,Alta Floresta D'Oeste,11,Sim,RO,11
1,1100023,Ariquemes,11,Sim,RO,11
2,1100031,Cabixi,11,Sim,RO,11
3,1100049,Cacoal,11,Não,RO,11
4,1100056,Cerejeiras,11,Sim,RO,11
...,...,...,...,...,...,...
5565,5222005,Vianópolis,52,Não,GO,52
5566,5222054,Vicentinópolis,52,Não,GO,52
5567,5222203,Vila Boa,52,Não,GO,52
5568,5222302,Vila Propício,52,Não,GO,52


In [41]:
df['ano']=2019

In [43]:
df = df.drop(['Cod UF'], axis=1)
df

,CodMun,NOME MUNIC,COD UF,MGOV01,UF,ano
0,1100015,Alta Floresta D'Oeste,11,Sim,RO,2019
1,1100023,Ariquemes,11,Sim,RO,2019
2,1100031,Cabixi,11,Sim,RO,2019
3,1100049,Cacoal,11,Não,RO,2019
4,1100056,Cerejeiras,11,Sim,RO,2019
...,...,...,...,...,...,...
5565,5222005,Vianópolis,52,Não,GO,2019
5566,5222054,Vicentinópolis,52,Não,GO,2019
5567,5222203,Vila Boa,52,Não,GO,2019
5568,5222302,Vila Propício,52,Não,GO,2019


In [46]:
df =  df.rename(columns={
                        'CodMun': 'id_municipio',
                        'NOME MUNIC':'nome_municipio',
                        'COD UF': 'cod_uf',
                        'UF':'sigla_uf',
                        'MGOV01':'existencia_legislacao_LAI',
})
df

,id_municipio,nome_municipio,cod_uf,existencia_legislacao_LAI,sigla_uf,ano
0,1100015,Alta Floresta D'Oeste,11,Sim,RO,2019
1,1100023,Ariquemes,11,Sim,RO,2019
2,1100031,Cabixi,11,Sim,RO,2019
3,1100049,Cacoal,11,Não,RO,2019
4,1100056,Cerejeiras,11,Sim,RO,2019
...,...,...,...,...,...,...
5565,5222005,Vianópolis,52,Não,GO,2019
5566,5222054,Vicentinópolis,52,Não,GO,2019
5567,5222203,Vila Boa,52,Não,GO,2019
5568,5222302,Vila Propício,52,Não,GO,2019


In [49]:
df=df[['ano', 'sigla_uf','cod_uf', 'nome_municipio', 'id_municipio', 'existencia_legislacao_LAI']]


In [52]:
df['existencia_legislacao_LAI']=np.where(df['existencia_legislacao_LAI']=='Recusa',np.nan,df['existencia_legislacao_LAI'])
df['existencia_legislacao_LAI']=np.where(df['existencia_legislacao_LAI']=='Não informou',np.nan,df['existencia_legislacao_LAI'])

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 0 to 5569
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   ano                        5570 non-null   int64 
 1   sigla_uf                   5570 non-null   object
 2   cod_uf                     5570 non-null   int64 
 3   nome_municipio             5570 non-null   object
 4   id_municipio               5570 non-null   int64 
 5   existencia_legislacao_LAI  5566 non-null   object
dtypes: int64(3), object(3)
memory usage: 304.6+ KB


Subindo para o GBQ

In [18]:
client = bigquery.Client()
dataset_ref = client.dataset('participacao_transparencia')

In [56]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
    bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação'),
    bigquery.SchemaField('cod_uf','INTEGER',description='Código do IBGE da UF'),
    bigquery.SchemaField('nome_municipio','STRING',description='Nome do município da observação'),
    bigquery.SchemaField('id_municipio','INTEGER',description='Identificador do município pelo IBGE'),
    bigquery.SchemaField('existencia_legislacao_LAI','STRING',description='Existência de legislação municipal para garantir direito de acesso à informação pública em conformidade com a Lei de Acesso à Informação'),
]

In [57]:
table_ref = dataset_ref.table('MUNIC_acesso_informacao_LAI')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=0e230911-78bc-4c6f-8c9e-f9e58fd35a92>